In [ ]:
!pip install scikit-image


In [ ]:
import pandas as pd
from pathlib import Path
import os
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt, image as img
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import (
    Conv2D,
    Conv2DTranspose,
    MaxPool2D,
    ReLU,
    Flatten,
    Dense,
    UpSampling2D,
    Dropout,
    Input,
    Concatenate,
    GlobalAveragePooling2D,
    GlobalMaxPooling2D,
)
from skimage.transform import resize
from tensorflow import Tensor
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.layers import Input
from tensorflow import Tensor

In [ ]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("mehradaria/covid19-lung-ct-scans")

# print("Path to dataset files:", path)

!rm -rf /root/.cache/kagglehub/datasets/mehradaria/covid19-lung-ct-scans

import kagglehub
path = kagglehub.dataset_download("mehradaria/covid19-lung-ct-scans")

print("New Path to dataset:", path)


100%|██████████| 1.02G/1.02G [00:47<00:00, 23.4MB/s]

Extracting files...


New Path to dataset: /root/.cache/kagglehub/datasets/mehradaria/covid19-lung-ct-scans/versions/1


In [ ]:
import shutil
import os

# Define original and new dataset paths
original_path = "/root/.cache/kagglehub/datasets/mehradaria/covid19-lung-ct-scans/versions/1"
dataset_path = "./COVID-19_Lung_CT_Scans"

# If the folder exists, remove it first to prevent conflicts
if os.path.exists(dataset_path):
    shutil.rmtree(dataset_path)

# Copy dataset files to the new location
shutil.copytree(original_path, dataset_path)

print(f"Dataset copied to: {dataset_path}")


Dataset copied to: ./COVID-19_Lung_CT_Scans


In [ ]:
import os

for root, dirs, files in os.walk(dataset_path):
    print(root)
    break  # Print only the top-level structure


./COVID-19_Lung_CT_Scans


In [ ]:
def load_raw_data():

    covid_path = "./COVID-19_Lung_CT_Scans/COVID-19_Lung_CT_Scans/COVID-19"
    non_covid_path = "./COVID-19_Lung_CT_Scans/COVID-19_Lung_CT_Scans/Non-COVID-19"


    covid_images = list(Path(covid_path).glob("*.png"))
    non_covid_images = list(Path(non_covid_path).glob("*.png"))

    return covid_images, non_covid_images

In [ ]:
covid_images, non_covid_images = load_raw_data()

print(len(covid_images))
print(len(non_covid_images))


7495
944


In [ ]:
data_dir="./COVID-19_Lung_CT_Scans/COVID-19_Lung_CT_Scans"
img_height,img_width=180,180
batch_size=32
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.4,
  subset="training",
  seed=123,
  label_mode="categorical",
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 8439 files belonging to 2 classes.
Using 5064 files for training.


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.4,
  subset="validation",
  seed=123,
  label_mode="categorical",
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 8439 files belonging to 2 classes.
Using 3375 files for validation.


In [ ]:
len(val_ds)


106

In [ ]:
val_batches = tf.data.experimental.cardinality(val_ds)

In [ ]:
test_ds = val_ds.take((1*val_batches) // 2)
val_ds = val_ds.skip((1*val_batches) // 2)

In [ ]:
class_names=train_ds.class_names
print(class_names)

['COVID-19', 'Non-COVID-19']


In [ ]:
for i in range(len(class_names)):
    print(class_names[i] ," " , i)

COVID-19   0
Non-COVID-19   1


In [ ]:
# Calculate the length of the training dataset
train_batches = len(train_ds)  # Number of batches in the training dataset
train_length = train_batches * batch_size  # Total number of elements in the training dataset

# Calculate the length of the validation dataset
val_batches = len(val_ds)  # Number of batches in the validation dataset
val_length = val_batches * batch_size  # Total number of elements in the validation dataset

# Calculate the length of the testing dataset
test_batches = len(test_ds)  # Number of batches in the testing dataset
test_length = test_batches * batch_size  # Total number of elements in the testing dataset

print("Length of the training dataset:", train_length)
print("Length of the validation dataset:", val_length)
print("Length of the testing dataset:", test_length)

Length of the training dataset: 5088
Length of the validation dataset: 1696
Length of the testing dataset: 1696


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.model_selection import train_test_split

# Image dimensions & batch size
img_height, img_width = 180, 180
batch_size = 32

# Data Augmentation for Training Set
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.4  # 40% data for validation + test
)

# Define dataset paths (UPDATE THESE)
data_dir = "./COVID-19_Lung_CT_Scans/COVID-19_Lung_CT_Scans"

# Load Train Dataset with Augmentation (60% Training Data)
train_ds = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

# Load Validation Dataset (40% Data)
val_ds = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
    shuffle=False  # Important for test set split
)

# Convert Validation Dataset to Numpy Arrays for Splitting
val_images, val_labels = [], []

for images, labels in val_ds:
    val_images.extend(images)
    val_labels.extend(labels)

    # Stop after full dataset is loaded
    if len(val_images) >= val_ds.samples:
        break

val_images = np.array(val_images)
val_labels = np.array(val_labels)

# Split Validation Data into Validation (50%) & Test (50%)
val_images, test_images, val_labels, test_labels = train_test_split(
    val_images, val_labels, test_size=0.5, random_state=42
)

# Convert Back to tf.data.Dataset
val_ds = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)

print("✅ Data Successfully Loaded!")


Found 5064 images belonging to 2 classes.
Found 3375 images belonging to 2 classes.
✅ Data Successfully Loaded!


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Get class distribution
class_counts = train_ds.classes  # Extract class labels
class_labels = np.unique(class_counts)

# Compute Class Weights
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=class_labels,
    y=class_counts
)

# Convert to Dictionary for Keras
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

print("✅ Computed Class Weights:", class_weights_dict)


✅ Computed Class Weights: {0: np.float64(0.5630420280186791), 1: np.float64(4.465608465608466)}


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.applications import ResNet50

# Define the Model
resnet_model = Sequential()

# Load Pretrained ResNet50 (Exclude Fully Connected Layer)
pretrained_model = ResNet50(
    include_top=False,
    input_shape=(img_height, img_width, 3),
    pooling="avg",
    weights="imagenet"
)

# Freeze Base Model
for layer in pretrained_model.layers:
    layer.trainable = False

resnet_model.add(pretrained_model)

# Add Custom Layers
resnet_model.add(Flatten())
resnet_model.add(BatchNormalization())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(BatchNormalization())
resnet_model.add(Dropout(0.5))
resnet_model.add(Dense(256, activation='relu'))
resnet_model.add(BatchNormalization())
resnet_model.add(Dropout(0.5))
resnet_model.add(Dense(128, activation='relu'))
resnet_model.add(BatchNormalization())
resnet_model.add(Dropout(0.5))
resnet_model.add(Dense(64, activation='relu'))
resnet_model.add(Dropout(0.5))
resnet_model.add(BatchNormalization())
resnet_model.add(Dense(2, activation='softmax'))  # 2 Classes

# Display Model Summary
resnet_model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 2048)                │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 2048)                │           8,192 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 512)                 │       1,049,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,821,442 (94.69 MB)

 Trainable params: 1,227,714 (4.68 MB)

 Non-trainable params: 23,593,728 (90.00 MB)

In [ ]:
from tensorflow.keras.optimizers import Adam

# Compile Model
resnet_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Train Model with Class Weights
epochs = 50
history = resnet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    class_weight=class_weights_dict  # Applying Class Weights
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 98s 503ms/step - accuracy: 0.5104 - loss: 0.9801 - val_accuracy: 0.8773 - val_loss: 0.4801
Epoch 2/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 66s 413ms/step - accuracy: 0.5695 - loss: 0.7572 - val_accuracy: 0.8512 - val_loss: 0.4480
Epoch 3/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 66s 414ms/step - accuracy: 0.6047 - loss: 0.6889 - val_accuracy: 0.8488 - val_loss: 0.4172
Epoch 4/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 65s 411ms/step - accuracy: 0.6592 - loss: 0.5497 - val_accuracy: 0.8370 - val_loss: 0.4158
Epoch 5/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 65s 406ms/step - accuracy: 0.7004 - loss: 0.4863 - val_accuracy: 0.8465 - val_loss: 0.3954
Epoch 6/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 65s 407ms/step - accuracy: 0.7371 - loss: 0.4181 - val_accuracy: 0.8613 - val_loss: 0.3671
Epoch 7/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 65s 408ms/step - accuracy: 0.7610 - loss: 0.4241 - val_accuracy: 0.8767 - val_loss: 0.3480
Epoch 8/50
159/159 ━━━━━━━━━━━━━━━━━━━━ 65s 412ms/step - accuracy: 0.7929 - loss: 0

In [ ]:
# Evaluate on Test Data
test_loss, test_acc = resnet_model.evaluate(test_ds)

print(f"✅ Test Accuracy: {test_acc:.4f}")


53/53 ━━━━━━━━━━━━━━━━━━━━ 11s 185ms/step - accuracy: 0.9341 - loss: 0.1811
✅ Test Accuracy: 0.9378


In [ ]:
# # Evaluate Model
# resnet_model.evaluate(test_ds)


53/53 ━━━━━━━━━━━━━━━━━━━━ 5s 84ms/step - accuracy: 0.9987 - loss: 0.0026


[0.002423187717795372, 0.9988207817077637]

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Get Predictions
y_pred_probs = resnet_model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert probabilities to class labels

# Get True Labels from test dataset
y_true = np.concatenate([np.argmax(labels.numpy(), axis=1) for _, labels in test_ds])

# Compute Precision, Recall, and F1-Score
precision = precision_score(y_true, y_pred, average="weighted")
recall = recall_score(y_true, y_pred, average="weighted")
f1 = f1_score(y_true, y_pred, average="weighted")

# Print Metrics
print(f"✅ Precision: {precision:.4f}")
print(f"✅ Recall: {recall:.4f}")
print(f"✅ F1-score: {f1:.4f}")


53/53 ━━━━━━━━━━━━━━━━━━━━ 12s 153ms/step
✅ Precision: 0.9366
✅ Recall: 0.9378
✅ F1-score: 0.9371


In [ ]:
# Compute Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Print Confusion Matrix
print("Confusion Matrix:")
print(conf_matrix)

# Print Classification Report
print("\nClassification Report:")
print(classification_report(y_true, y_pred))


Confusion Matrix:
[[1451   48]
 [  57  132]]

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.97      0.97      1499
           1       0.73      0.70      0.72       189

    accuracy                           0.94      1688
   macro avg       0.85      0.83      0.84      1688
weighted avg       0.94      0.94      0.94      1688

